### Import libs

In [1]:
import os
import torch
from torch_geometric.data import HeteroData
from langchain_core.tools import tool
from typing import Set, Tuple, Dict
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

/mnt/drive/hoangbaoan/repos/llm-ids/LMTracker/Phase2/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Reformat ouptut data

In [2]:
test_file_path = "/mnt/drive/hoangbaoan/repos/llm-ids/LMTracker/Phase2/data/test_paths_FULL.csv"
train_file_path = "/mnt/drive/hoangbaoan/repos/llm-ids/LMTracker/Phase2/data/train_paths_FULL.csv"

# Import to pandas df
test_df = pd.read_csv(test_file_path)
train_df = pd.read_csv(train_file_path)

In [3]:
test_df

,split,seed,method,threshold,score,label_true,label_pred,is_fp,is_fn,path,node_0,node_1,node_2
0,TEST 1 - graph_sample_benign_split,0,FULL_ENSEMBLE,0.385267,0.119151,0,0,0,0,"[4881, 20778, 66]",4881,20778,66
1,TEST 1 - graph_sample_benign_split,0,FULL_ENSEMBLE,0.385267,0.143543,0,0,0,0,"[3436, 19701, 3436]",3436,19701,3436
2,TEST 1 - graph_sample_benign_split,0,FULL_ENSEMBLE,0.385267,0.220768,0,0,0,0,"[4358, 15716, 99]",4358,15716,99
3,TEST 1 - graph_sample_benign_split,0,FULL_ENSEMBLE,0.385267,0.257728,0,0,0,0,"[6128, 15674, 10]",6128,15674,10
4,TEST 1 - graph_sample_benign_split,0,FULL_ENSEMBLE,0.385267,0.114550,0,0,0,0,"[2359, 18107, 43]",2359,18107,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52165,TEST 1 - graph_sample_benign_split,9,FULL_ENSEMBLE,0.397865,0.482175,1,1,0,0,"[9641, 17849, 3400]",9641,17849,3400
52166,TEST 1 - graph_sample_benign_split,9,FULL_ENSEMBLE,0.397865,0.415676,1,1,0,0,"[9641, 22467, 8885]",9641,22467,8885
52167,TEST 1 - graph_sample_benign_split,9,FULL_ENSEMBLE,0.397865,0.162856,1,0,0,1,"[13379, 16623, 8]",13379,16623,8
52168,TEST 1 - graph_sample_benign_split,9,FULL_ENSEMBLE,0.397865,0.468092,1,1,0,0,"[9641, 17316, 6292]",9641,17316,6292


In [4]:
train_df

,split,seed,method,threshold,score,label_true,label_pred,is_fp,is_fn,path,node_0,node_1,node_2
0,TEST 1 - graph_sample_benign_split,0,FULL_ENSEMBLE,0.385267,NaN,0,NaN,0,0,"[5575, 15480, 6059]",5575,15480,6059
1,TEST 1 - graph_sample_benign_split,0,FULL_ENSEMBLE,0.385267,NaN,0,NaN,0,0,"[66, 26102, 202]",66,26102,202
2,TEST 1 - graph_sample_benign_split,0,FULL_ENSEMBLE,0.385267,NaN,0,NaN,0,0,"[11198, 24849, 1872]",11198,24849,1872
3,TEST 1 - graph_sample_benign_split,0,FULL_ENSEMBLE,0.385267,NaN,0,NaN,0,0,"[6686, 15480, 833]",6686,15480,833
4,TEST 1 - graph_sample_benign_split,0,FULL_ENSEMBLE,0.385267,NaN,0,NaN,0,0,"[10296, 15480, 10537]",10296,15480,10537
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,TEST 1 - graph_sample_benign_split,9,FULL_ENSEMBLE,0.397865,NaN,0,NaN,0,0,"[115, 16615, 6172]",115,16615,6172
99996,TEST 1 - graph_sample_benign_split,9,FULL_ENSEMBLE,0.397865,NaN,0,NaN,0,0,"[2148, 16674, 1]",2148,16674,1
99997,TEST 1 - graph_sample_benign_split,9,FULL_ENSEMBLE,0.397865,NaN,0,NaN,0,0,"[7684, 15480, 11955]",7684,15480,11955
99998,TEST 1 - graph_sample_benign_split,9,FULL_ENSEMBLE,0.397865,NaN,0,NaN,0,0,"[9012, 21744, 14107]",9012,21744,14107


In [5]:
# Takes only `label_true`, `label_pred`, `node_0`, `node_1` and `node_2` from the dataframes
test_df = test_df[['seed', 'label_true', 'label_pred', 'node_0', 'node_1', 'node_2']]
test_df

,seed,label_true,label_pred,node_0,node_1,node_2
0,0,0,0,4881,20778,66
1,0,0,0,3436,19701,3436
2,0,0,0,4358,15716,99
3,0,0,0,6128,15674,10
4,0,0,0,2359,18107,43
...,...,...,...,...,...,...
52165,9,1,1,9641,17849,3400
52166,9,1,1,9641,22467,8885
52167,9,1,0,13379,16623,8
52168,9,1,1,9641,17316,6292


In [6]:
train_df = train_df[['seed', 'label_true', 'label_pred', 'node_0', 'node_1', 'node_2']]
train_df['label_pred'] = train_df['label_true']
train_df

/tmp/ipykernel_786983/2554869929.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['label_pred'] = train_df['label_true']


,seed,label_true,label_pred,node_0,node_1,node_2
0,0,0,0,5575,15480,6059
1,0,0,0,66,26102,202
2,0,0,0,11198,24849,1872
3,0,0,0,6686,15480,833
4,0,0,0,10296,15480,10537
...,...,...,...,...,...,...
99995,9,0,0,115,16615,6172
99996,9,0,0,2148,16674,1
99997,9,0,0,7684,15480,11955
99998,9,0,0,9012,21744,14107


In [7]:
train_df = train_df[train_df['seed'] == 5]
test_df = test_df[test_df['seed'] == 5]

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [8]:
train_df

,seed,label_true,label_pred,node_0,node_1,node_2
0,5,0,0,1076,20189,15134
1,5,0,0,5383,15480,11119
2,5,0,0,9888,23695,202
3,5,0,0,1243,15425,3053
4,5,0,0,3603,17990,3603
...,...,...,...,...,...,...
9995,5,0,0,9672,15920,212
9996,5,0,0,9235,15479,3284
9997,5,0,0,66,21700,1
9998,5,0,0,1,16678,6171


In [9]:
test_df

,seed,label_true,label_pred,node_0,node_1,node_2
0,5,0,0,12,16039,12
1,5,0,1,4997,22423,4997
2,5,0,0,6920,20535,6920
3,5,0,0,12470,24091,8
4,5,0,0,10,23737,10
...,...,...,...,...,...,...
5212,5,1,1,9641,21658,2328
5213,5,1,1,9641,16521,9562
5214,5,1,1,9641,15467,11922
5215,5,1,1,9641,17849,9712


### Load embeddings

In [10]:
import pickle
out_mal_test_embeddings = pickle.load(open("/mnt/drive/hoangbaoan/repos/llm-ids/LMTracker/Phase2/data/model_CUC_20251120175600/out_mal_test.pkl", "rb"))
print(type(out_mal_test_embeddings))
print(len(out_mal_test_embeddings))
print(type(list(out_mal_test_embeddings.values())[0]))
print(len(list(out_mal_test_embeddings.values())[0]))

<class 'dict'>
245
<class 'tuple'>
128


In [11]:
out_mal_train_val_embeddings = pickle.load(open("/mnt/drive/hoangbaoan/repos/llm-ids/LMTracker/Phase2/data/model_CUC_20251120175600/out_mal_train_val.pkl", "rb"))
print(type(out_mal_train_val_embeddings))
print(len(out_mal_train_val_embeddings))
print(type(list(out_mal_train_val_embeddings.values())[0]))
print(len(list(out_mal_train_val_embeddings.values())[0]))

<class 'dict'>
229
<class 'tuple'>
128


In [12]:
out_normal_embeddings = pickle.load(open("/mnt/drive/hoangbaoan/repos/llm-ids/LMTracker/Phase2/data/model_CUC_20251120175600/out_normal.pkl", "rb"))
print(type(out_normal_embeddings))
print(len(out_normal_embeddings))

<class 'dict'>
588958


In [13]:
path_embeddings = pickle.load(open("/mnt/drive/hoangbaoan/repos/llm-ids/LMTracker/Phase2/data/model_CUC_20251120175600/path_embedding.pkl", "rb"))
print(type(path_embeddings))
print(len(path_embeddings))

<class 'dict'>
31152


In [14]:
# Taking the union of all embeddings
all_embeddings = out_mal_test_embeddings | out_mal_train_val_embeddings | out_normal_embeddings | path_embeddings
print(len(all_embeddings))

615415


In [15]:
print(all_embeddings[(5575,15480,6059)])

(-0.00036501672002486885, 0.004345093388110399, -0.007321050390601158, -0.0028715557418763638, -0.000767064280807972, -0.0025533013977110386, 0.006852022372186184, 0.004699792712926865, -0.00027518635033629835, -0.0009995207656174898, 0.0002516162348911166, 0.00216465862467885, -0.0022183971013873816, 0.002553182654082775, 0.0010819006711244583, 0.0006469878717325628, 0.0034028682857751846, 0.004396400414407253, -0.008489139378070831, -0.0005771444411948323, -0.026079537346959114, -0.011133982799947262, 0.0012585906079038978, 0.006689500063657761, -0.006438404321670532, 0.0016479844925925136, 0.17409662902355194, -0.004254284780472517, -0.05586545914411545, 0.00042784842662513256, 0.007130858488380909, -0.0010466361418366432, -0.0008610840886831284, 0.7468998432159424, -0.004494038876146078, 0.008026449009776115, 0.023933228105306625, -0.008766449987888336, 0.0008609829237684608, 0.0017740167677402496, 0.001655506668612361, -0.010655287653207779, -1.810694084269926e-05, -0.010333368554

### Merge embeddings into dataframe

In [16]:
# Now, merge them into a single table
def get_embedding(row):
    return all_embeddings.get(tuple(row))

# Add embeddings to both dataframes
train_df['embedding'] = train_df[['node_0', 'node_1', 'node_2']].apply(get_embedding, axis=1)
test_df['embedding'] = test_df[['node_0', 'node_1', 'node_2']].apply(get_embedding, axis=1)

/tmp/ipykernel_786983/707172332.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['embedding'] = train_df[['node_0', 'node_1', 'node_2']].apply(get_embedding, axis=1)
/tmp/ipykernel_786983/707172332.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['embedding'] = test_df[['node_0', 'node_1', 'node_2']].apply(get_embedding, axis=1)


In [17]:
train_df

,seed,label_true,label_pred,node_0,node_1,node_2,embedding
0,5,0,0,1076,20189,15134,"(0.012800626456737518, -0.004206163343042135, ..."
1,5,0,0,5383,15480,11119,"(0.001360858790576458, -1.0459565601195209e-05..."
2,5,0,0,9888,23695,202,"(0.0074762082658708096, 0.016626553609967232, ..."
3,5,0,0,1243,15425,3053,"(-0.003926246892660856, 0.008524230681359768, ..."
4,5,0,0,3603,17990,3603,"(0.00839211791753769, 0.0006869967910461128, 0..."
...,...,...,...,...,...,...,...
9995,5,0,0,9672,15920,212,"(-0.0018333191983401775, 0.018805647268891335,..."
9996,5,0,0,9235,15479,3284,"(0.005529539659619331, 0.0010887137614190578, ..."
9997,5,0,0,66,21700,1,"(-0.0016085882671177387, -0.005039993673563003..."
9998,5,0,0,1,16678,6171,"(0.002121415687724948, 0.011293587274849415, 0..."


In [18]:
test_df

,seed,label_true,label_pred,node_0,node_1,node_2,embedding
0,5,0,0,12,16039,12,"(0.012532912194728851, 0.01285327598452568, 0...."
1,5,0,1,4997,22423,4997,"(0.0011252912227064371, -0.0011719162575900555..."
2,5,0,0,6920,20535,6920,"(0.00462216604501009, 0.009731483645737171, 3...."
3,5,0,0,12470,24091,8,"(0.006704645697027445, 0.013978617265820503, 0..."
4,5,0,0,10,23737,10,"(-0.0004738150746561587, -0.000278733670711517..."
...,...,...,...,...,...,...,...
5212,5,1,1,9641,21658,2328,"(0.04362139478325844, -0.007252213545143604, -..."
5213,5,1,1,9641,16521,9562,"(-0.04563647508621216, 0.011731836013495922, 0..."
5214,5,1,1,9641,15467,11922,"(0.0042697894386947155, 0.019935954362154007, ..."
5215,5,1,1,9641,17849,9712,"(-0.04499203711748123, 0.024102818220853806, 0..."


In [19]:
train_none_count = train_df['embedding'].isnull().sum()
test_none_count = test_df['embedding'].isnull().sum()

print(f"Null embeddings on train_df: {train_none_count}")
print(f"Null embeddings on test_df: {test_none_count}")

Null embeddings on train_df: 0
Null embeddings on test_df: 0


In [20]:
train_df.head()

,seed,label_true,label_pred,node_0,node_1,node_2,embedding
0,5,0,0,1076,20189,15134,"(0.012800626456737518, -0.004206163343042135, ..."
1,5,0,0,5383,15480,11119,"(0.001360858790576458, -1.0459565601195209e-05..."
2,5,0,0,9888,23695,202,"(0.0074762082658708096, 0.016626553609967232, ..."
3,5,0,0,1243,15425,3053,"(-0.003926246892660856, 0.008524230681359768, ..."
4,5,0,0,3603,17990,3603,"(0.00839211791753769, 0.0006869967910461128, 0..."


In [21]:
train_df.dtypes

seed           int64
label_true     int64
label_pred     int64
node_0         int64
node_1         int64
node_2         int64
embedding     object
dtype: object

### Remove user's offset

In [22]:
def remove_offset(row):
    return int(row) - 15425 # in embeddings_output_explore

train_df['node_1'] = train_df[['node_1']].apply(remove_offset, axis=1)
test_df['node_1'] = test_df[['node_1']].apply(remove_offset, axis=1)

/tmp/ipykernel_786983/3427418003.py:2: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(row) - 15425 # in embeddings_output_explore
/tmp/ipykernel_786983/3427418003.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['node_1'] = train_df[['node_1']].apply(remove_offset, axis=1)
/tmp/ipykernel_786983/3427418003.py:2: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(row) - 15425 # in embeddings_output_explore
/tmp/ipykernel_786983/3427418003.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

### Dump embeddings into pickle files

In [23]:
pickle.dump(train_df, open("/mnt/drive/hoangbaoan/repos/llm-ids/LMTracker/Phase2/data/train_paths_seed5_wembeddings.pkl", "wb"),protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump(test_df, open("/mnt/drive/hoangbaoan/repos/llm-ids/LMTracker/Phase2/data/test_paths_seed5_wembeddings.pkl", "wb"),protocol=pickle.HIGHEST_PROTOCOL)

#### Load in again to test

In [24]:
train_df = pickle.load(open("/mnt/drive/hoangbaoan/repos/llm-ids/LMTracker/Phase2/data/train_paths_seed5_wembeddings.pkl", "rb"))
train_df.dtypes

seed           int64
label_true     int64
label_pred     int64
node_0         int64
node_1         int64
node_2         int64
embedding     object
dtype: object

In [25]:
test_df = pickle.load(open("/mnt/drive/hoangbaoan/repos/llm-ids/LMTracker/Phase2/data/test_paths_seed5_wembeddings.pkl", "rb"))
test_df.dtypes

seed           int64
label_true     int64
label_pred     int64
node_0         int64
node_1         int64
node_2         int64
embedding     object
dtype: object

In [26]:
train_df

,seed,label_true,label_pred,node_0,node_1,node_2,embedding
0,5,0,0,1076,4764,15134,"(0.012800626456737518, -0.004206163343042135, ..."
1,5,0,0,5383,55,11119,"(0.001360858790576458, -1.0459565601195209e-05..."
2,5,0,0,9888,8270,202,"(0.0074762082658708096, 0.016626553609967232, ..."
3,5,0,0,1243,0,3053,"(-0.003926246892660856, 0.008524230681359768, ..."
4,5,0,0,3603,2565,3603,"(0.00839211791753769, 0.0006869967910461128, 0..."
...,...,...,...,...,...,...,...
9995,5,0,0,9672,495,212,"(-0.0018333191983401775, 0.018805647268891335,..."
9996,5,0,0,9235,54,3284,"(0.005529539659619331, 0.0010887137614190578, ..."
9997,5,0,0,66,6275,1,"(-0.0016085882671177387, -0.005039993673563003..."
9998,5,0,0,1,1253,6171,"(0.002121415687724948, 0.011293587274849415, 0..."


In [27]:
test_df

,seed,label_true,label_pred,node_0,node_1,node_2,embedding
0,5,0,0,12,614,12,"(0.012532912194728851, 0.01285327598452568, 0...."
1,5,0,1,4997,6998,4997,"(0.0011252912227064371, -0.0011719162575900555..."
2,5,0,0,6920,5110,6920,"(0.00462216604501009, 0.009731483645737171, 3...."
3,5,0,0,12470,8666,8,"(0.006704645697027445, 0.013978617265820503, 0..."
4,5,0,0,10,8312,10,"(-0.0004738150746561587, -0.000278733670711517..."
...,...,...,...,...,...,...,...
5212,5,1,1,9641,6233,2328,"(0.04362139478325844, -0.007252213545143604, -..."
5213,5,1,1,9641,1096,9562,"(-0.04563647508621216, 0.011731836013495922, 0..."
5214,5,1,1,9641,42,11922,"(0.0042697894386947155, 0.019935954362154007, ..."
5215,5,1,1,9641,2424,9712,"(-0.04499203711748123, 0.024102818220853806, 0..."


In [28]:
print(f"negative user row in train_df = {(train_df['node_1'] < 0).sum()}")
print(f"negative user row in test_df = {(test_df['node_1'] < 0).sum()}")

negative user row in train_df = 0
negative user row in test_df = 0


In [29]:
test_df['from'] = "autoencoder prediction"
train_df['from'] = "ground truth"
df_merged = pd.concat([train_df, test_df], ignore_index=True, sort=False)
df_merged


,seed,label_true,label_pred,node_0,node_1,node_2,embedding,from
0,5,0,0,1076,4764,15134,"(0.012800626456737518, -0.004206163343042135, ...",ground truth
1,5,0,0,5383,55,11119,"(0.001360858790576458, -1.0459565601195209e-05...",ground truth
2,5,0,0,9888,8270,202,"(0.0074762082658708096, 0.016626553609967232, ...",ground truth
3,5,0,0,1243,0,3053,"(-0.003926246892660856, 0.008524230681359768, ...",ground truth
4,5,0,0,3603,2565,3603,"(0.00839211791753769, 0.0006869967910461128, 0...",ground truth
...,...,...,...,...,...,...,...,...
15212,5,1,1,9641,6233,2328,"(0.04362139478325844, -0.007252213545143604, -...",autoencoder prediction
15213,5,1,1,9641,1096,9562,"(-0.04563647508621216, 0.011731836013495922, 0...",autoencoder prediction
15214,5,1,1,9641,42,11922,"(0.0042697894386947155, 0.019935954362154007, ...",autoencoder prediction
15215,5,1,1,9641,2424,9712,"(-0.04499203711748123, 0.024102818220853806, 0...",autoencoder prediction


In [30]:
# Convert tuples to a list of lists/numpy array
embeddings_list = df_merged['embedding'].tolist()
embeddings_array = np.array(embeddings_list)

# Apply Min-Max Scaler
scaler = StandardScaler()
scaled_embeddings_array = scaler.fit_transform(embeddings_array)

# Convert back to tuples
df_merged['embedding'] = [tuple(row) for row in scaled_embeddings_array]

df_merged

,seed,label_true,label_pred,node_0,node_1,node_2,embedding,from
0,5,0,0,1076,4764,15134,"(0.5259035035359626, -0.5622622908350943, 3.17...",ground truth
1,5,0,0,5383,55,11119,"(-0.1794086620178113, -0.330011154494062, 0.11...",ground truth
2,5,0,0,9888,8270,202,"(0.19762962297258843, 0.5909225959107908, -0.5...",ground truth
3,5,0,0,1243,0,3053,"(-0.5053820618086453, 0.14242246172857678, -0....",ground truth
4,5,0,0,3603,2565,3603,"(0.2540994953266966, -0.29140379802372224, 1.2...",ground truth
...,...,...,...,...,...,...,...,...
15212,5,1,1,9641,6233,2328,"(2.426139897034724, -0.7308749142032261, -2.11...",autoencoder prediction
15213,5,1,1,9641,1096,9562,"(-3.077001716284376, 0.31997789052878417, 0.15...",autoencoder prediction
15214,5,1,1,9641,42,11922,"(-6.024911787179869e-05, 0.7741128605419226, 0...",autoencoder prediction
15215,5,1,1,9641,2424,9712,"(-3.0372692713121596, 1.004767577246367, 0.743...",autoencoder prediction


In [31]:
pickle.dump(df_merged, open("/mnt/drive/hoangbaoan/repos/llm-ids/LMTracker/Phase2/data/seed5_df.pkl", "wb"),protocol=pickle.HIGHEST_PROTOCOL)